In [57]:
from sklearn.pipeline import Pipeline

from sklearn.datasets import fetch_openml

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd


In [58]:
X1,y = fetch_openml(data_id=43592,return_X_y=True,as_frame=True,
                   parser='auto')

In [59]:
X = X1[['text']].copy()
y = X1['label'].copy()

In [60]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X['text'],y,random_state=42,shuffle=True,test_size=.2)

In [61]:
y

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object

In [62]:
model = Pipeline(
    steps=[
        ('binarizer',CountVectorizer(ngram_range=(1,1),analyzer='word',max_df=.8,min_df=.0005,binary=True)),
        ('learner',RandomForestClassifier(random_state=42))
    ]
).fit(Xtrain,ytrain)

In [63]:
yfit = model.predict(Xtrain)
ypred = model.predict(Xtest)

In [64]:
from sklearn.metrics import accuracy_score

In [65]:
accuracy_score(ytest,ypred)

0.9779005524861878

In [66]:
Xtest.values[0]

'Will Trump pull a Brexit times ten? What would it take, beyond WikiLeaks, to bring the Clinton (cash) machine down? Will Hillary win and then declare WWIII against her Russia/Iran/Syria axis of evil? Will the Middle East totally explode? Will the pivot to Asia totally implode? Will China be ruling the world by 2025?nAmidst so many frenetic fragments of geopolitical reality precariously shored against our ruins, the temptation is irresistible to hark back to the late, great, deconstructionist master Jean Baudrillard. During the post-mod 1980s it was hip to be Baudrillardian to the core; his America, originally published in France in 1986, should still be read today as the definitive metaphysical/geological/cultural Instagram of Exceptionalistan.nBy the late 1990s, at the end of the millennium, two years before 9/11  that seminal before and after event  Baudrillard was already stressing how we live in a black market maze. Now, its a black market paroxysm.nGlobal multitudes are subjected

In [67]:
ytest

1357    FAKE
2080    FAKE
2718    FAKE
812     FAKE
4886    FAKE
        ... 
1512    FAKE
57      REAL
6092    REAL
3403    REAL
292     REAL
Name: label, Length: 1267, dtype: object

In [69]:
new_text = pd.Series(
    ['Today in Brooklyn, a man was gunned down by two armed female attackers. The assailants are still at large. Anyone with information should contact the NYPD.']
)

In [70]:
model.predict(new_text)

array(['FAKE'], dtype=object)

In [ ]:
new_text = pd.Series(
    [
        """
        Federal prosecutors on Tuesday accused Rep. George Santos of stealing donors’ identities and running up thousands of dollars in fraudulent charges on their credit cards.
        The 10 new charges against Santos also include allegations that the New York Republican embezzled cash from his company and conspired with his former campaign treasurer to falsify donation totals in order to hit fundraising targets set by national Republicans, among other offenses.
        """
    ]
)